In [1]:
import ee
ee.Authenticate()


Successfully saved authorization token.


In [2]:
ee.Initialize()
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import geopandas as gpd

In [3]:
addevent_can = r"C:\FLUJOS_Local\shapefiles_csdap\Impact_Nebraska_dissolved.shp" #''2019-04-22', '2019-05-05''


# This code downloads S2 images by event's different dates, same day imgs are mosaic together
Also displays the last img downloaded(for reference how to display on map)

In [5]:


scene_name = 'addevent_can'  # Replace with your scene name
scene = addevent_can  # Replace with the path to your shapefile

scene_geemap = geemap.shp_to_ee(scene).geometry()

collection_S2 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(scene_geemap)
    .filterDate('2022-10-31', '2022-11-15')
)
def get_date(image):
    return ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')


dates_list = collection_S2.map(lambda img: img.set('date', get_date(img))).aggregate_array('date').getInfo()
distinct_dates = set(dates_list)
distinct_dates = list(distinct_dates)

print("Dates with images in the collection:", dates_list)
print(distinct_dates)

for date in distinct_dates:
    print(date)
    # Filter the ImageCollection based on the current date
    
    images_same_date = collection_S2.filterDate(date, ee.Date(date).advance(1, 'day'))
    print(images_same_date.aggregate_array('system:id').getInfo())
    
    bands = ['VV', 'VH']
    # # Mosaic the images from the same date
    mosaic_S2 = images_same_date.mosaic().select(bands)
    # # user_input = input("Press Enter to continue...")
    
    folder = r"C:\FLUJOS_Local\s_1\\" + scene_name + "\\" + date
    if not os.path.exists(folder): os.makedirs(folder)
    

    # file_path = folder + "\\" + mosaic_S2.aggregate_array('system:id').getInfo()[0].split('/')[-2] + '_' + mosaic_S2.aggregate_array('system:id').getInfo()[0].split('/')[-1]+".tif"
    file_path = folder + "\\" + date +".tif"
    geemap.download_ee_image(mosaic_S2, file_path, region= scene_geemap, crs="EPSG:4326", scale=10)
    print("------------------")
    
    


Dates with images in the collection: ['2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-05', '2022-11-08', '2022-11-08', '2022-11-08', '2022-11-08', '2022-11-08', '2022-11-08', '2022-11-08', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-10', '2022-11-13', '2022-11-13', '2022-11-13', '2022-11-13', '2022-11-13', '2022-11-13', '2022-11-13']
['2022-11-08', '2022-11-13', '2022-11-03', '2022-11-05', '2022-10-31', '2022-11-10']
2022-11-08
['COPERNICUS/S2_SR_HARMONIZED/20221108T172529_20221108T173019_T14TPK', 'COPERNICUS/S2_SR_HARMONIZED/20221108T172529_20221108T173019_T14TPL', 'COPERNICUS

Consider adjusting `region`, `scale` and/or `dtype` to reduce the 2022-11-08.tif download size (raw: 9.92 GB).


2022-11-08.tif: |          | 0.00/9.92G (raw) [  0.0%] in 00:00 (eta:     ?)

In [6]:
gdf = gpd.read_file(scene)
bbox = gdf.total_bounds
center_lat = (bbox[1] + bbox[3]) / 2
center_lng = (bbox[0] + bbox[2]) / 2
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.0
}
Map = geemap.Map()

# Add the image and shapefile to the map
Map.addLayer(mosaic_S2, vis_params, 'First Image')
# Map.addLayer(ee_object, {}, 'Shapefile')

# Set the map center and zoom level according to the shapefile
Map.setCenter(center_lng, center_lat, 10)

# Display the map
Map

Map(center=[46.25347769503688, -72.7400196125835], controls=(WidgetControl(options=['position', 'transparent_b…